In [1]:
#Instalamos la libreria de STREAMLIT
%pip install streamlit

Note: you may need to restart the kernel to use updated packages.


In [2]:
#Instalamos la libreria de STREAMLIT
!pip install streamlit

In [3]:
#Instalamos la libreria de PLOTLY
%pip install plotly

Note: you may need to restart the kernel to use updated packages.


In [4]:
#Para instalar npm en visual studio
#1.Desde Google escribir node.js
#2. Instalar la versión más recomendada
! npm install localtunnel


up to date, audited 23 packages in 3s

3 packages are looking for funding
  run `npm fund` for details

2 high severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


In [5]:
%pip install statsmodels

Note: you may need to restart the kernel to use updated packages.


In [6]:

# app_airbnb.py — Dashboard para Inside Airbnb (Stockholm)
# Ejecuta con:  streamlit run app_airbnb.py

import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, confusion_matrix, accuracy_score, precision_score
import pydeck as pdk
from scipy.optimize import curve_fit

st.set_page_config(page_title="Airbnb Stockholm • Analytics", page_icon="🏠", layout="wide")

# ----------------------------
# 1) Carga y preparación de datos
# ----------------------------
@st.cache_data(show_spinner=False)
def load_data(path:str) -> pd.DataFrame:
    df = pd.read_csv(path)
    # Normaliza price a numérico si es texto tipo "$1,234.00"
    if df['price'].dtype == 'object':
        df['price'] = (df['price']
                       .astype(str)
                       .str.replace(r'[^0-9\.-]', '', regex=True)
                       .replace('', np.nan)
                       .astype(float))
    # Extrae bathrooms_float desde bathrooms_text (e.g., "1 bath", "1.5 shared baths")
    if 'bathrooms_text' in df.columns and 'bathrooms' in df.columns:
        # Conserva bathrooms si ya es float; si no, intenta parsear desde bathrooms_text
        if not np.issubdtype(df['bathrooms'].dtype, np.number):
            df['bathrooms'] = pd.to_numeric(df['bathrooms'], errors='coerce')
    if 'bathrooms_text' in df.columns:
        bt = (df['bathrooms_text'].astype(str)
              .str.extract(r'([0-9]*\.?[0-9]+)')[0]
              .astype(float))
        df['bathrooms_parsed'] = bt

    # Convierte columnas dicotómicas útiles
    for col in ['host_is_superhost','instant_bookable','has_availability','host_has_profile_pic','host_identity_verified']:
        if col in df.columns and df[col].dtype == 'object':
            df[col] = df[col].map({'t':1,'f':0,'True':1,'False':0}).astype('float64')

    # Crea bandera de Entire home/apt si room_type existe
    if 'room_type' in df.columns:
        df['is_entire_home'] = (df['room_type'] == 'Entire home/apt').astype('int8')

    # Asegura tipos numéricos en variables claves si existen
    numeric_candidates = [
        'accommodates','bedrooms','beds','minimum_nights','maximum_nights',
        'availability_30','availability_60','availability_90','availability_365',
        'number_of_reviews','number_of_reviews_ltm','number_of_reviews_l30d',
        'review_scores_rating','review_scores_accuracy','review_scores_cleanliness',
        'review_scores_checkin','review_scores_communication','review_scores_location',
        'review_scores_value','estimated_occupancy_l365d','estimated_revenue_l365d',
        'calculated_host_listings_count','calculated_host_listings_count_entire_homes',
        'calculated_host_listings_count_private_rooms','calculated_host_listings_count_shared_rooms',
        'latitude','longitude','price','bathrooms','bathrooms_parsed'
    ]
    for c in numeric_candidates:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors='coerce')

    # Filtrado mínimo para mapa: quitar lat/long nulos
    if {'latitude','longitude'}.issubset(df.columns):
        df = df[~df['latitude'].isna() & ~df['longitude'].isna()].copy()

    return df

df = load_data("estocolmofinal.csv")

# ----------------------------
# 2) Listas de variables seguras (según tus columnas reales)
# ----------------------------
# Categóricas "manejables": objetos con número de categorías moderado
cat_cols = []
for c in df.columns:
    if df[c].dtype == 'object':
        nun = df[c].nunique(dropna=True)
        if 2 <= nun <= 40:
            cat_cols.append(c)

# Algunas muy útiles si existen
preferred_cats = [c for c in [
    'neighbourhood_cleansed','room_type','property_type','host_response_time',
    'host_is_superhost','instant_bookable'
] if c in df.columns]
cat_cols = list(dict.fromkeys(preferred_cats + cat_cols))  # prioriza las comunes

# Numéricas
num_cols = [c for c in df.select_dtypes(include=[np.number]).columns if df[c].notna().sum() > 0]
preferred_nums = [c for c in [
    'price','accommodates','bedrooms','beds','bathrooms','bathrooms_parsed',
    'minimum_nights','maximum_nights','availability_365','number_of_reviews',
    'review_scores_rating','estimated_occupancy_l365d','estimated_revenue_l365d'
] if c in df.columns]
# Ordena con preferencia primero
num_cols = list(dict.fromkeys(preferred_nums + num_cols))

# ----------------------------
# 3) Sidebar
# ----------------------------
st.sidebar.title("🏠 Airbnb Stockholm")
page = st.sidebar.selectbox(
    "Vista",
    ["Resumen & Filtros", "Exploración de categorías", "Regresión lineal", "Regresión no lineal", "Regresión logística"]
)

# Filtros comunes
with st.sidebar.expander("Filtros (aplican en 'Resumen & Filtros')"):
    # Filtro por room_type si existe
    if 'room_type' in df.columns:
        room_opts = ["(Todos)"] + sorted(df['room_type'].dropna().unique().tolist())
        room_sel = st.selectbox("Room type", room_opts)
    else:
        room_sel = "(Todos)"

    # Filtro por vecindario
    if 'neighbourhood_cleansed' in df.columns:
        n_opts = ["(Todos)"] + sorted(df['neighbourhood_cleansed'].dropna().unique().tolist())
        n_sel = st.selectbox("Neighbourhood", n_opts)
    else:
        n_sel = "(Todos)"

    # Rango de precio
    if 'price' in df.columns and df['price'].notna().any():
        pmin, pmax = float(np.nanmin(df['price'])), float(np.nanmax(df['price']))
        price_range = st.slider("Rango de precio", min_value=0.0, max_value=float(pmax), value=(0.0, float(min(pmax, 5000))))  # límites razonables

# Aplica filtros a una copia
dff = df.copy()
if page == "Resumen & Filtros":
    if 'room_type' in dff.columns and room_sel != "(Todos)":
        dff = dff[dff['room_type'] == room_sel]
    if 'neighbourhood_cleansed' in dff.columns and n_sel != "(Todos)":
        dff = dff[dff['neighbourhood_cleansed'] == n_sel]
    if 'price' in dff.columns and dff['price'].notna().any():
        dff = dff[(dff['price'] >= price_range[0]) & (dff['price'] <= price_range[1])]

# ----------------------------
# 4) Página: Resumen & Filtros
# ----------------------------
if page == "Resumen & Filtros":
    st.title("Resumen | Filtros y mapa interactivo")

    # KPI cards
    k1, k2, k3, k4 = st.columns(4)
    with k1:
        st.metric("Listado(s) activos", value=int(dff.shape[0]))
    with k2:
        if 'price' in dff.columns and dff['price'].notna().any():
            st.metric("Precio mediano", value=f"{np.nanmedian(dff['price']):.0f}")
        else:
            st.metric("Precio mediano", value="N/D")
    with k3:
        m = 'accommodates'
        st.metric("Capacidad mediana", value=int(np.nanmedian(dff[m])) if m in dff.columns else "N/D")
    with k4:
        if 'review_scores_rating' in dff.columns:
            st.metric("Rating promedio", value=f"{np.nanmean(dff['review_scores_rating']):.1f}")
        else:
            st.metric("Rating promedio", value="N/D")

    # Mapa
    if {'latitude','longitude'}.issubset(dff.columns):
        tooltip_cols = [c for c in ['name','price','room_type','neighbourhood_cleansed','accommodates'] if c in dff.columns]
        st.subheader("🗺️ Mapa de listados")
        st.pydeck_chart(pdk.Deck(
            map_style=None,
            initial_view_state=pdk.ViewState(
                latitude=float(dff['latitude'].median()),
                longitude=float(dff['longitude'].median()),
                zoom=10,
                pitch=0,
            ),
            layers=[
                pdk.Layer(
                    "ScatterplotLayer",
                    data=dff,
                    get_position='[longitude, latitude]',
                    get_radius=30,
                    pickable=True,
                )
            ],
            tooltip={"text": "\n".join([f"{c}: {{{c}}}" for c in tooltip_cols])} if tooltip_cols else None,
        ))

    # Distribuciones rápidas
    st.subheader("Distribuciones")
    figcols = st.columns(3)
    dist_candidates = [c for c in ['price','accommodates','number_of_reviews','review_scores_rating'] if c in dff.columns]
    for i, c in enumerate(dist_candidates[:3]):
        with figcols[i]:
            st.plotly_chart(px.histogram(dff, x=c, nbins=40, title=f"Distribución de {c}"), use_container_width=True)

# ----------------------------
# 5) Página: Exploración de categorías
# ----------------------------
if page == "Exploración de categorías":
    st.title("Exploración de categorías")
    if not cat_cols:
        st.info("No se detectaron columnas categóricas adecuadas.")
    else:
        cat_sel = st.selectbox("Variable categórica", cat_cols)
        tabf = dff[cat_sel].value_counts(dropna=False).reset_index()
        tabf.columns = [cat_sel, 'frecuencia']

        c1, c2 = st.columns(2)
        with c1:
            st.plotly_chart(px.bar(tabf, x=cat_sel, y='frecuencia', title=f"Frecuencia por {cat_sel}"), use_container_width=True)
        with c2:
            st.plotly_chart(px.pie(tabf, names=cat_sel, values='frecuencia', title=f"Proporción por {cat_sel}"), use_container_width=True)

        st.dataframe(tabf)

# ----------------------------
# 6) Página: Regresión lineal
# ----------------------------
if page == "Regresión lineal":
    st.title("Regresión lineal (precio y más)")

    if not num_cols or len(num_cols) < 2:
        st.warning("No hay suficientes variables numéricas para modelar.")
    else:
        y_var = st.selectbox("Variable objetivo (Y)", options=num_cols, index=num_cols.index('price') if 'price' in num_cols else 0)
        x_var = st.selectbox("Variable independiente (X)", options=[c for c in num_cols if c != y_var])

        # Simple
        dmod = df[[x_var, y_var]].dropna()
        if dmod.empty:
            st.error("No hay datos suficientes tras eliminar NAs.")
        else:
            X = dmod[[x_var]].values
            y = dmod[y_var].values
            lr = LinearRegression().fit(X, y)
            yhat = lr.predict(X)
            r2 = lr.score(X, y)
            st.metric("R² (simple)", f"{r2:.3f}")
            st.plotly_chart(px.scatter(dmod, x=x_var, y=y_var, title=f"{y_var} vs {x_var}")                                .add_traces([go.Scatter(x=dmod[x_var], y=yhat, name="Ajuste", mode="lines")]), use_container_width=True)

        # Múltiple
        x_multi = st.multiselect("Variables independientes (múltiple)", options=[c for c in num_cols if c != y_var],
                                 default=[c for c in ['accommodates','bedrooms','beds'] if c in num_cols])
        if x_multi:
            dmod2 = df[x_multi + [y_var]].dropna()
            if not dmod2.empty:
                X2 = dmod2[x_multi].values
                y2 = dmod2[y_var].values
                lr2 = LinearRegression().fit(X2, y2)
                r2m = lr2.score(X2, y2)
                st.metric("R² (múltiple)", f"{r2m:.3f}")
                # Importancias relativas vía coeficientes (estandarizados aproximados)
                scaler = StandardScaler()
                X2s = scaler.fit_transform(X2)
                lr2s = LinearRegression().fit(X2s, y2)
                coefs = pd.DataFrame({"feature": x_multi, "coef_estandarizado": lr2s.coef_}).sort_values("coef_estandarizado", key=abs, ascending=False)
                st.subheader("Peso (aprox.) de variables")
                st.dataframe(coefs)
            else:
                st.info("Sin datos para el modelo múltiple tras eliminar NAs.")

# ----------------------------
# 7) Página: Regresión no lineal
# ----------------------------
if page == "Regresión no lineal":
    st.title("Regresión no lineal por segmento")
    if len(num_cols) < 2:
        st.warning("Se requieren al menos dos variables numéricas.")
    else:
        y_var = st.selectbox("Variable objetivo (Y)", options=num_cols, index=num_cols.index('price') if 'price' in num_cols else 0)
        x_var = st.selectbox("Variable independiente (X)", options=[c for c in num_cols if c != y_var])
        model_type = st.selectbox("Modelo", ["Cuadrático", "Exponencial (a*exp(-b*x)+c)"])

        # Segmentación por room_type si existe; si no, por is_entire_home si existe; si no, sin segmentar
        if 'room_type' in df.columns:
            seg_vals = df['room_type'].dropna().unique().tolist()[:2]
            if len(seg_vals) < 2:
                seg_vals = [seg_vals[0], seg_vals[0]]
            segA, segB = seg_vals[0], seg_vals[1]
            dA = df[df['room_type'] == segA][[x_var, y_var]].dropna()
            dB = df[df['room_type'] == segB][[x_var, y_var]].dropna()
            labelA, labelB = f"Segmento: {segA}", f"Segmento: {segB}"
        elif 'is_entire_home' in df.columns:
            dA = df[df['is_entire_home'] == 1][[x_var, y_var]].dropna()
            dB = df[df['is_entire_home'] == 0][[x_var, y_var]].dropna()
            labelA, labelB = "Entire home/apt", "No entire home"
        else:
            dA = df[[x_var, y_var]].dropna()
            dB = df[[x_var, y_var]].dropna()
            labelA, labelB = "Todos (A)", "Todos (B)"

        def fit_and_plot(data, label):
            if data.empty:
                return None, None, None
            x = data[x_var].values
            y = data[y_var].values

            if model_type == "Cuadrático":
                def f(x,a,b,c): return a*x**2 + b*x + c
            else:
                def f(x,a,b,c): return a*np.exp(-b*x)+c

            try:
                params, _ = curve_fit(f, x, y, maxfev=20000)
                yhat = f(x, *params)
                r2 = r2_score(y, yhat)
                fig = px.scatter(data, x=x_var, y=y_var, title=f"{label} — R²={r2:.3f}")
                fig.add_trace(go.Scatter(x=x, y=yhat, mode="lines", name="Ajuste"))
                return r2, params, fig
            except Exception as e:
                st.info(f"No se pudo ajustar el modelo para {label}: {e}")
                return None, None, None

        c1, c2 = st.columns(2)
        with c1:
            r2A, pA, fA = fit_and_plot(dA, labelA)
            if fA is not None: st.plotly_chart(fA, use_container_width=True)
        with c2:
            r2B, pB, fB = fit_and_plot(dB, labelB)
            if fB is not None: st.plotly_chart(fB, use_container_width=True)

# ----------------------------
# 8) Página: Regresión logística
# ----------------------------
if page == "Regresión logística":
    st.title("Regresión logística (clasificación)")
    # Variable Y por defecto: host_is_superhost (si existe), si no is_entire_home
    y_candidates = [c for c in ['host_is_superhost','is_entire_home','instant_bookable','has_availability'] if c in df.columns]
    if not y_candidates:
        st.warning("No hay variable dicotómica adecuada (ej. host_is_superhost).")
    else:
        y_var = st.selectbox("Variable objetivo (Y, dicotómica)", options=y_candidates)
        X_options = [c for c in num_cols if c not in [y_var]]
        default_X = [c for c in ['price','accommodates','bedrooms','beds','number_of_reviews','review_scores_rating'] if c in X_options] or X_options[:3]
        X_sel = st.multiselect("Variables independientes (X numéricas)", options=X_options, default=default_X)

        if X_sel:
            dmod = df[X_sel + [y_var]].dropna()
            if dmod.empty:
                st.error("No hay datos suficientes tras eliminar NAs.")
            else:
                X = dmod[X_sel].values
                y = dmod[y_var].values
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y if len(np.unique(y))>1 else None)

                scaler = StandardScaler()
                X_train = scaler.fit_transform(X_train)
                X_test  = scaler.transform(X_test)

                clf = LogisticRegression(max_iter=2000)
                clf.fit(X_train, y_train)
                y_pred = clf.predict(X_test)

                acc = accuracy_score(y_test, y_pred)
                st.metric("Exactitud", f"{acc:.3f}")
                # Matriz de confusión
                cm = confusion_matrix(y_test, y_pred)
                labels = [str(i) for i in np.unique(y)]
                heat = go.Figure(data=go.Heatmap(z=cm, x=labels, y=labels, hoverinfo="z"))
                heat.update_layout(title="Matriz de confusión", xaxis_title="Predicho", yaxis_title="Real", width=500, height=500)
                st.plotly_chart(heat, use_container_width=False)

                # Precisión por clase (si binaria)
                if len(labels) == 2:
                    p0 = precision_score(y_test, y_pred, pos_label=float(labels[0]))
                    p1 = precision_score(y_test, y_pred, pos_label=float(labels[1]))
                    st.write(f"Precisión clase {labels[0]}: {p0:.3f}")
                    st.write(f"Precisión clase {labels[1]}: {p1:.3f}")
        else:
            st.info("Selecciona al menos una X.")

2025-10-23 00:31:52.501 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-23 00:31:52.505 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-10-23 00:31:52.509 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-10-23 00:31:52.864 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-23 00:31:54.280 
  command:

    streamlit run C:\Users\Jose Vera\AppData\Roaming\Python\Python313\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-10-23 00:31:54.282 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-23 00:31:54.283 Thread 'MainThread': missing ScriptRunContext! This warning can b